In [4]:
import sys
import os
sys.path.append('../')
from torch.utils.data import DataLoader

import stepselector.zarr_dataloader as zdl

In [5]:
server_mount = '/home/blair/server/herd_hover'

In [7]:
data_folder = os.path.join(server_mount, 'zebra_movement_data')
tracks_directory = os.path.join(data_folder, 'tracks_zarr_validated')

step_length = 5
offsets = list(range(0, step_length))

In [8]:
# Compute global stats for variable standardizations

stats_5m = zdl.compute_global_stats_for_observed(
    zarr_root=tracks_directory,
    step_length=step_length,
    offsets_forward=[0, 1, 2, 3, 4],
    offsets_backward=[0, 1, 2, 3, 4],
    obs_to_process=None,
)

In [ ]:
# Build dataset

dataset_5m = zdl.StepSelectionDataset(
    zarr_root=tracks_directory,
    step_length=step_length,
    offsets_forward=[0, 1, 2, 3, 4],
    offsets_backward=[0, 1, 2, 3, 4],
    global_stats=stats_5m,
    S=10,          # number of sims per observed (≤ 40)
    N=10,          # context length
    obs_to_process=None,
    features_observed = [
                "angle_to_observer",
                "delta_observer_dist",
                "step_speed_mps",
                "ground_slope_deg",
                "ground_class_name",
                "trail_probability",
                "viewshed_vis",
                "social_density",
                "social_visible_count",
                "nn_distance_m",
                "nn_visible",
                "nn_individual_ID",
                "nn_species",
                "observation_ID"
                "age",
                "individual_ID",
                "species",
            ]
)

print(len(dataset_5m))
sample = dataset_5m[0]
print(sample["observed"].shape, sample["context"].shape, sample["simulated"].shape)
print(sample["meta"])

In [ ]:
# Load dataset

loader = DataLoader(dataset_5m, batch_size=64, shuffle=True)

for batch in loader:
    obs = batch["observed"]      # (B, F_obs)
    ctx = batch["context"]       # (B, N, F_ctx)
    sim = batch["simulated"]     # (B, S, F_sim)
    meta = batch["meta"]
    # ... feed into your model ...
    break